In [1]:
%reload_ext autoreload
%autoreload 2

import warnings

import polars as pl

# from sklearn import set_config

warnings.filterwarnings("ignore")

_ = pl.Config.set_tbl_rows(5)
_ = pl.Config.set_tbl_cols(20)

# set_config(transform_output="polars")

In [2]:
import polars as pl

train_df = pl.read_csv(
    "https://huggingface.co/datasets/inria-soda/tabular-benchmark/resolve/main/reg_cat/diamonds.csv"
)

train_df = train_df.with_columns(
    pl.col("cut").cast(pl.String).cast(pl.Categorical),
    pl.col("color").cast(pl.String).cast(pl.Categorical),
    pl.col("clarity").cast(pl.String).cast(pl.Categorical),
)

train_df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
f64,cat,cat,cat,f64,f64,f64,f64,f64,f64
0.23,"""2""","""1""","""3""",61.5,55.0,3.95,3.98,2.43,5.78996
0.21,"""3""","""1""","""2""",59.8,61.0,3.89,3.84,2.31,5.78996
0.23,"""1""","""1""","""4""",56.9,65.0,4.05,4.07,2.31,5.793014
0.29,"""3""","""5""","""5""",62.4,58.0,4.2,4.23,2.63,5.814131
0.31,"""1""","""6""","""3""",63.3,58.0,4.34,4.35,2.75,5.817111


In [14]:
import polars.selectors as cs

# from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_validate
from skrub import AggTarget, Cleaner, tabular_learner

# Clean
c = Cleaner()
train_df: pl.DataFrame = c.fit_transform(train_df)  # type: ignore

# Target Encoding
X = train_df.drop("price")
y = train_df.select("price")

for col in X.select(cs.categorical()).columns:
    aggregator = AggTarget(
        main_key=col,
        operations=[
            "mean",
        ],
    )

    aggregator.fit(X, y)

    X = aggregator.transform(X)

# lgbm = LGBMRegressor(force_row_wise=True)
# model = tabular_learner(lgbm)
model = tabular_learner("regressor")
results = cross_validate(model, X, y)  # type: ignore

print(f"R2: {results['test_score'].mean()}")

R2: 0.8274079052028913


In [15]:
# import optuna
# from sklearn.model_selection import cross_val_score


# def objective(trial):
#     # Define parameter ranges for optimization
#     params = {"regressor__n_estimators": trial.suggest_int("n_estimators", 100, 1000)}

#     # Create a new pipeline with the suggested parameters
#     pipeline.set_params(**params)

#     # Fit and return the score
#     pipeline.fit(X, y)
#     cvs = cross_val_score(pipeline, X, y, cv=5).mean()  # type: ignore
#     return cvs


# optuna.logging.set_verbosity(optuna.logging.WARNING)
# study = optuna.create_study()
# study.optimize(objective, n_trials=10, n_jobs=-1)  # type: ignore

# print(study.best_params)
# print(study.best_value)
